In [1]:
import pdb
import numpy as np
import gcp.bigquery as bq
import gcp.storage as storage
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
try:
   import cPickle as pickle
except:
   import pickle
EST_PICKLE_FILENAME = 'baseline_final_estimator.pkl'

# Put all categorical data first for easier implementation of One Hot Encoding.
fields_str = """
gap	day_in_week	weather_1_slots_ago	weather_2_slots_ago	weather_3_slots_ago	busy_time	
tj_level1_1_slots_ago	tj_level2_1_slots_ago	tj_level3_1_slots_ago	tj_level4_1_slots_ago	
tj_level1_2_slots_ago	tj_level2_2_slots_ago	tj_level3_2_slots_ago	tj_level4_2_slots_ago	
tj_level1_3_slots_ago	tj_level2_3_slots_ago	tj_level3_3_slots_ago	tj_level4_3_slots_ago	
temperature_1_slots_ago	pm25_1_slots_ago	
temperature_2_slots_ago	pm25_2_slots_ago	
temperature_3_slots_ago	pm25_3_slots_ago	
gap_1_slots_ago	sum_price_1_slots_ago	
gap_2_slots_ago	sum_price_2_slots_ago	
gap_3_slots_ago	sum_price_3_slots_ago	
f1	f11	f11_1	f11_2	f11_3	f11_4	f11_5	f11_6	f11_7	
f11_8	f13_4	f13_8	f14	f14_1	f14_10	f14_2	f14_3	f14_6	f14_8	f15	f15_1	
f15_2	f15_3	f15_4	f15_6	f15_7	f15_8	f16	f16_1	f16_10	f16_11	f16_12	f16_3	
f16_4	f16_6	f17	f17_2	f17_3	f17_4	f17_5	f19	f19_1	f19_2	f19_3	f19_4	f1_1	
f1_10	f1_11	f1_2	f1_3	f1_4	f1_5	f1_6	f1_7	f1_8	f20	f20_1	f20_2	
f20_4	f20_5	f20_6	f20_7	f20_8	f20_9	f21_1	f21_2	f22	f22_1	f22_2	f22_3	
f22_4	f22_5	f23	f23_1	f23_2	f23_3	f23_4	f23_5	f23_6	f24	f24_1	f24_2	f24_3	
f25	f25_1	f25_3	f25_7	f25_8	f25_9	f2_1	f2_10	f2_11	f2_12	f2_13	f2_2	
f2_4	f2_5	f2_6	f2_7	f2_8	f3_1	f3_2	f3_3	f4	f4_1	f4_10	f4_11	
f4_13	f4_14	f4_16	f4_17	f4_18	f4_2	f4_3	f4_5	f4_6	f4_7	f4_8	f4_9	
f5	f5_1	f5_3	f5_4	f6	f6_1	f6_2	f6_4	f7	f8	f8_1	f8_2	f8_3	f8_4	
f8_5
"""
fields = map(lambda x: x.strip(), fields_str.split('\t'))
features = fields[1:]

# Use this instead of len(features) since this variable can change
# e.g. when one hot encoding is used and/or new features are added.
n_features = len(features)

# Scorer Creation (MAPE)

In [8]:
def mape(y, predictions):
#   num_timeslots = 43
#   num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  y = y.astype(float)
  predictions = predictions.astype(float)
  return np.sum(np.absolute((y-predictions)/y)) / (y.shape[0]*predictions.shape[0])

# from keras import backend as K

# def mape(y, predictions):
#   return K.mean(K.abs(y-predictions/K.clip(K.abs(y), K.epsilon(), np.inf)), axis=-1)

mape_scorer = make_scorer(mape, greater_is_better=False)

Testing MAPE

In [9]:
from sklearn.linear_model import LogisticRegression

predictions = np.array([1.0, 2.0, 3.0, 4.0]).astype('float32')
y = np.array([1.0, 2.0, 3.0, 4.0]).astype('float32')

# Should return 0.0
print mape(y, predictions)

# Should return higher score
predictions = np.array([1.0, 2.0, 2.0, 3.0]).astype('float32')
print(mape(y, predictions))

# Should return highest score
predictions = np.array([1000.0, 22.0, 11.0, 31.0]).astype('float32')
print(mape(y, predictions))

# est = LogisticRegression()
# X = np.random.rand(10,4)
# y = X.sum(axis=1)
# est.fit(X, y)
# predictions = est.predict(X)
# print(mape(y, predictions))

0.0
0.583333333333
1018.41666667


# Feature Selection

In [10]:
%%sql --module q_all

SELECT *, HASH(CAST(district_id AS STRING) +timeslot) AS hash_value,
  IF(ABS(HASH(CAST(district_id AS STRING) + timeslot)) % 2 == 1, 'True', 'False')
    AS included_in_sample, IF(timeofday_slot >= 50 AND timeofday_slot <= 53, 1, 0) AS busy_time
FROM [datalab-projects-1331:xjk_algo_comp.gaps]
WHERE gap > 0
LIMIT 5000

# The above query randomizes its outputs.

In [11]:
query = bq.Query(q_all)
tableresult = query.results()

all_data = np.zeros((tableresult.length, len(fields)))
print 'there are {} rows'.format(tableresult.length)
for rcounter, row in enumerate(tableresult):
  for fcounter, field in enumerate(fields):
    all_data[rcounter, fcounter] = row[field]
  if rcounter % 5000 == 0:
    print 'processed {} rows'.format(rcounter)
all_data_original = np.copy(all_data)

there are 5000 rows
processed 0 rows


In [12]:
# This chunk does further wrangling to dataset to produce training and test sets.

# Useful code to check NaN and Inf values. This is needed since these values would
# cause "Input contains NaN, infinity or a value too large for dtype('float32')
# errors when left unchecked.
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

# Impute all NaN with numbers (not sure what to replace inf yet)
all_data[np.isnan(all_data)] = 0
# all_data[np.isinf(all_data)] = 0

# See that NaN and Inf values replaced
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

# Split the data into train and test sets.
data_size = all_data.shape[0]
training_size = data_size * 80/100
indices = np.random.permutation(data_size)
training_idx, test_idx = indices[:training_size], indices[training_size:]
all_data_train, all_data_test = all_data[training_idx,:], all_data[test_idx,:]

data_train = all_data_train[:,1:]
targets_train = all_data_train[:,0]
data_test = all_data_test[:,1:]
targets_test = all_data_test[:,0]
data_train_original = np.copy(data_train)
data_test_original = np.copy(data_test)

Checkinf for NaN and Inf
np.nan= (array([  25,   25,   32, ..., 4993, 4993, 4993]), array([24, 25,  6, ..., 21, 22, 23]))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= nan
Checkinf for NaN and Inf
np.nan= (array([], dtype=int64), array([], dtype=int64))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= 461563.0


In [13]:
from sklearn.preprocessing import OneHotEncoder
# one_hot = OneHotEncoder(categorical_features=[0, 1, 14, 17, 20], n_values='auto')
one_hot = OneHotEncoder(categorical_features=[0, 1, 2, 3], sparse=False)
one_hot.fit(data_train)

OneHotEncoder(categorical_features=[0, 1, 2, 3], dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [14]:
one_hot.n_values_

array([ 7, 10, 10, 10])

In [15]:
one_hot.feature_indices_

array([ 0,  7, 17, 27, 37])

In [16]:
data_train = one_hot.transform(data_train_original).todense()
data_test = one_hot.transform(data_test_original).todense()
n_features = data_train.shape[1]
print 'new number of features: {}'.format(n_features)

new number of features: 192


# Building and Testing Algorithm(s)

In [17]:
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.sparse import coo_matrix, hstack

class CustomRegressor(BaseEstimator, RegressorMixin):
  def __init__(self):
    pass

  def fit(self, X, y):
#     self.classes_, indices = np.unique(["foo", "bar", "foo"],
#                                     return_inverse=True)
#     self.majority_ = np.argmax(np.bincount(indices))
    return self

  def predict(self, X):
    # 56: gap_1_slots_ago
    # 58: gap_2_slots_ago
    # 60: gap_3_slots_ago
#     X = X.tocsr()
#     v1 = coo_matrix(np.asmatrix(np.ones(X.shape[0])).T)
    v1 = np.asmatrix(np.ones(X.shape[0]))
    v2 = np.asmatrix((X[:, 23]*0.65+X[:, 25]*0.25+X[:, 27]*0.15)/2)
    predictions = np.asarray(np.concatenate((v1, v2), axis=0).max(axis=0))
    
    return predictions
  
custom_est = CustomRegressor()
custom_est.fit(data_train_original, data_test_original)
custom_predictions = custom_est.predict(data_test_original)
print(mape(targets_test, custom_predictions))

423.136508905


In [18]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA

steps = [
  ('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], n_values='auto')),
#   ('impute', Imputer(0)),
#   ('feature_selection', SelectKBest(f_classif)),
  ('pca', PCA(n_components=120)),
  ('estimate', DecisionTreeRegressor())
]

est = Pipeline(steps)

params = {
#   'one_hot__n_values': [7, 10, 20],
#   "feature_selection__k": [i for i in range(1, n_features - 1)]
  'estimate__max_features': [i for i in range(110, 120)],
#   'estimate__learning_rate': [0.1, 0.5, 1, 5, 10],
#   'estimate__n_estimators': [i for i in range(110, 120, 2)],
#   'estimate__loss': ['linear', 'square', 'exponential']
}
# cross_validation_iter = StratifiedShuffleSplit(y=targets_train, test_size=0.3,
#                                                random_state=RANDOM_STATE, n_iter=10)
# search_params = RandomizedSearchCV(
#   estimator=est,
#   param_distributions=params,
#   cv=5,
#   scoring=mape_scorer,
#   n_jobs=2,
#   verbose=1
# )

search_params = GridSearchCV(
  estimator=est,
  param_grid=params,
  cv=5,
  scoring=mape_scorer,
  n_jobs=5,
  verbose=3
)

search_params.fit(data_train_original, targets_train)
print(search_params.grid_scores_)
print(search_params.best_params_)
print(search_params.best_score_)
search_params.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=110 ......................................
[CV] estimate__max_features=111 ......................................
[CV] estimate__max_features=111 ......................................
[CV] estimate__max_features=111 ......................................
[CV] estimate__max_features=111 ......................................


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

...........................................................

Test data's prediction MAPE score:

In [ ]:
final_est = search_params.best_estimator_
test_predictions = final_est.predict(data_test_original)
print(mape(targets_test, test_predictions))

In [ ]:
pickle.dump(final_est, open(EST_PICKLE_FILENAME, "w") )

Run "Process Final Test Data With Final Algorithm" to use pickled final algorithm against final test data to produce csv required by this competition.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
# Just testing Imputer. Turns out somehow Imputer causes number of features reduced, weird.

# imputer = Imputer()
est = DecisionTreeRegressor(max_features=len(features))

data_train_i = np.copy(data_train)
print(data_train.shape)
print(data_train[0:10])
# data_train_i = imputer.fit_transform(data_train)
data_train_i[np.isnan(data_train_i)] = 0
data_train_i.astype('float32')
print(data_train_i.shape)
print(data_train_i[0:10])
est.fit(data_train_i, targets_train)
predictions = est.predict(data_test)
print(mape(data_test, predictions, targets_test))

# Results

- DecisionTreeRegressor + all features (31951 training data): 2.141
- DecisionTreeRegressor + all features (44202 training data): 3.065
- DecisionTreeRegressor + all + one hot encoded features (81832 training data): 5.726
- AdaBoostRegressor + all + one hot encoded features + PCA (102592 training data): 